# World Carbon Pricing Database Report
## Automated Report with Mappings


In [ ]:
import yaml
import pandas as pd
from glob import glob
from IPython.display import Image, display

with open('../scripts/report_config.yaml') as f:
    config = yaml.safe_load(f)

# Load data
wcpd_df = pd.concat([pd.read_csv(f) for f in glob("../_dataset/data/CO2/*.csv")])
ecp_df = pd.concat([pd.read_csv(f) for f in glob("../_dataset/ecp/ipcc/*.csv")])
dfs = {'WCPD_CO2': wcpd_df, 'ECP_IPCC': ecp_df}

# Helper
def get_data_point(label, jurisdiction, year, config, dfs):
    meta = config['mappings'][label]
    if meta.get('static', False):
        return jurisdiction
    df = dfs[meta['file']]
    if 'jurisdiction' in df.columns:
        df = df[df['jurisdiction'] == jurisdiction]
    if 'year' in df.columns and year is not None:
        df = df[df['year'] == year]
    col = meta['column']
    op = meta['operation']
    if op == 'mean':
        return df[col].mean()
    elif op == 'max':
        return df[col].max()
    elif op == 'sum':
        return df[col].sum()
    elif op == 'weighted_average':
        wcol = meta['weight_column']
        return (df[col] * df[wcol]).sum() / df[wcol].sum()
    elif op == 'list':
        return df[col].dropna().unique().tolist()
    elif op == 'percent_global':
        total = df[col].sum()
        grand_total = dfs[meta['file']][col].sum()
        return 100 * total / grand_total
    elif op == 'count_unique':
        return df[df[col] == meta['filter_value']]['jurisdiction'].nunique()
    elif op == 'count_both':
        counts = dfs[meta['file']].groupby('jurisdiction')[col].nunique()
        return counts[counts >= 2].count()
    else:
        raise ValueError(f"Unknown op {op}")


## 1️⃣ Jurisdiction Fact Sheets

In [ ]:
for jurisdiction in config['jurisdictions']:
    latest_year = get_data_point('latest_year', jurisdiction, None, config, dfs)
    avg_price = get_data_point('average_price', jurisdiction, latest_year, config, dfs)
    print(f"{jurisdiction} ({latest_year}): Average price = {avg_price:.2f} USD/tCO2")
    display(Image(f"../_output/charts/{jurisdiction.lower().replace(' ', '_')}_price_trend.png"))


## 2️⃣ Cross-jurisdiction Summary

In [ ]:
g_avg = get_data_point('global_average_price', '', None, config, dfs)
print(f"Global Average Price: {g_avg:.2f} USD/tCO2")
display(Image("../_output/charts/global_price_trend.png"))
